In [1]:
%%capture
!pip install cmath
!pip install numpy
!pip install scipy

In [2]:
import cmath
import numpy as np
from scipy.integrate import quad

In [3]:
# Compute Heston price of a European contingent claim via Fourier transform and numerical integration
#     T                      : time to expiration
#     F                      : underlying forward price (settles at T)
#     D                      : discount factor (matures at T)
#     v0                     : initial spot variance
#     vBar                   : Heston long-term variance
#     lambda_                : Heston spot variance speed of mean reversion
#     eta                    : Heston volatility of volatility
#     rho                    : Heston correlation between underlying and spot variance brownian motion increments
#     payoffFourierTransform : Fourier transform of the European payoff which is being priced
#     beta                   : imaginary frequency shift parameter (omega => omega - i * beta)
def HestonFourierPrice(T, F, D, v0, vBar, lambda_, eta, rho, payoffFourierTransform, beta):
    f = np.log(F)
    rhoEta = rho * eta
    eta2 = eta ** 2
    lambdaOverEta2 = lambda_ / eta2

    def CharacteristicFunctionPrice(z):
        tmp1 = lambda_ - rhoEta * z
        d = np.sqrt(tmp1 ** 2 - eta2 * z * (z - 1))
        tmp2 = tmp1 - d
        tmp3 = np.exp(-d * T)
        g = tmp2 / (tmp1 + d)
        tmp4 = g * tmp3 - 1
        C = lambdaOverEta2 * (tmp2 * T - 2 * np.log(tmp4 / (g - 1)))
        D = (tmp2 / eta2) * ((tmp3 - 1) / tmp4)
        return np.exp(z * f + C * vBar + D * v0)

    def Integrand(omega):
        # 1j represents the unit imaginary number "i" in Python code
        return np.real(payoffFourierTransform(omega - 1j * beta) * CharacteristicFunctionPrice(beta + 1j * omega))

    # Integrand should always be an even function, thus take twice the integral over positive half line
    return D * quad(Integrand, 0, np.inf, full_output = 1)[0] / np.pi

In [4]:
# Generates the Fourier transform of a European call option payoff with strike K
#     K : call option strike price
def CallPayoffFourierTransform(K):
    k = np.log(K)

    def Ghat(omega): # Fourier transform of G(x) = max(exp(x) - K, 0)
        iomega = 1j * omega
        return np.exp((1 - iomega) * k) / (iomega * (iomega - 1))

    return Ghat

In [5]:
# Sample Heston Fourier pricing (should return 1.88...)
HestonFourierPrice(
    T = 0.25,
    F = 600,
    D = 0.99,
    v0 = 0.010201,
    vBar = 0.019,
    lambda_ = 6.21,
    eta = 0.61,
    rho = -0.7,
    payoffFourierTransform = CallPayoffFourierTransform(K = 630),
    beta = 4)

1.8831037503074106